### 1. Global Settings

In [3]:
import sys
sys.path.append("../../")
import json
import os
import surprise
import papermill as pm
import pandas as pd
import shutil
import subprocess
import time
import yaml
from tempfile import TemporaryDirectory

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, precision_at_k, ndcg_at_k
from reco_utils.nni.nni_utils import check_experiment_status, check_stopped, check_metrics_written, get_trials
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

tmp_dir = TemporaryDirectory()

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Surprise version: 1.0.6


### 2. Prepare Dataset
1. Download data and split into training, validation and test sets
2. Store the data sets to a local directory.

In [4]:
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'
SURPRISE_READER = 'ml-100k'
TMP_DIR = tmp_dir.name

In [5]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

4.93MB [00:01, 4.35MB/s]                            


,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [6]:
train, validation, test = python_random_split(data, [0.7, 0.15, 0.15])

In [7]:
DATA_DIR = os.path.join(TMP_DIR, 'aml_data') 
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

### 3. Prepare Hyperparameter Tuning 

We now prepare a training script [svd_training_nni.py](../../reco_utils/nni/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as precision, NDCG, RMSE.
We define the arguments of the script and the search space for the hyperparameters. All the parameter values will be passed to our training script.

In [8]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model"
PRIMARY_METRIC = 'precision_at_k'
RATING_METRICS = ['rmse']
RANKING_METRICS = ['precision_at_k', 'ndcg_at_k']  
USERCOL = 'userID'
ITEMCOL = 'itemID'
RECOMMEND_SEEN = False
K = 10
RANDOM_STATE = 0
VERBOSE = True
NUM_EPOCHS = 30
BIASED = True

script_params = " ".join([
    '--datastore', DATA_DIR,
    '--train-datapath', TRAIN_FILE_NAME,
    '--validation-datapath', VAL_FILE_NAME,
    '--surprise-reader', SURPRISE_READER,
    '--rating-metrics', " ".join(RATING_METRICS),
    '--ranking-metrics', " ".join(RANKING_METRICS),
    '--usercol', USERCOL,
    '--itemcol', ITEMCOL,
    '--k', str(K),
    '--random-state', str(RANDOM_STATE),
    '--epochs', str(NUM_EPOCHS),
    '--primary-metric', PRIMARY_METRIC
])

if BIASED:
    script_params += ' --biased'
if VERBOSE:
    script_params += ' --verbose'
if RECOMMEND_SEEN:
    script_params += ' --recommend-seen'

In [9]:
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overriden by the other lr_ and reg_ parameters

hyper_params = {
    'n_factors': {"_type": "choice", "_value": [10, 50, 100, 150, 200]},
    'init_mean': {"_type": "uniform", "_value": [-0.5, 0.5]},
    'init_std_dev': {"_type": "uniform", "_value": [0.01, 0.2]},
    'lr_bu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_bi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_pu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_qi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'reg_bu': {"_type": "uniform", "_value": [1e-6, 1]},
    'reg_bi': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_pu': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_qi': {"_type": "uniform", "_value": [1e-6, 1]}
}

In [11]:
with open(os.path.join(TMP_DIR, 'search_space_svd.json'), 'w') as fp:
    json.dump(hyper_params, fp)

We also create a yaml file for the configuration of the trials and the tuning algorithm to be used (in this experiment we use the [TPE tuner](https://nni.readthedocs.io/en/latest/hyperoptTuner.html)). The tuning trials will be executed locally on a [Standard_D16_v3 virtual machine](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-general#dv3-series-1) (16 vcpus, 64 GB memory).

In [12]:
config = {
    'authorName': 'default',
    'experimentName': 'surprise_svd',
    'trialConcurrency': 8,
    'maxExecDuration': '1h',
    'maxTrialNum': 100,
    'trainingServicePlatform': 'local',
    # The path to Search Space
    'searchSpacePath': 'search_space_svd.json',
    'useAnnotation': False,
    'tuner': {
        'builtinTunerName': 'TPE',
        'classArgs': {
            #choice: maximize, minimize
            'optimize_mode': 'maximize'
        }
    },
    # The path and the running command of trial
    'trial':  {
      'command': 'python3 svd_training.py' + " " + script_params,
      'codeDir': os.path.abspath('../../reco_utils/nni/'),
      'gpuNum': 0
    }
}
 
with open(os.path.join(TMP_DIR, 'config_svd.yml'), 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

### 4. Execute NNI Trials

The conda environment comes with NNI installed, which includes the command line tool `nnictl` for controlling and getting information about NNI experiments. <br>
To start the NNI tuning trials from the command line, execute the following command: <br>
`nnictl create --config <path of config_svd.yml>` <br>
In the cell below, we call this command programmatically. <br>
You can see the progress of the experiment by using the URL links output by the above command.

![](https://recodatasets.blob.core.windows.net/images/nn1.png)

![](https://recodatasets.blob.core.windows.net/images/nn2.png)

![](https://recodatasets.blob.core.windows.net/images/nn3.png)

In [13]:
config_path = os.path.join(TMP_DIR, 'config_svd.yml')
os.environ['PATH'] = sys.prefix + '/bin:' + os.environ['PATH']
proc = subprocess.run([sys.prefix + '/bin/nnictl', 'create', '--config', config_path])
assert proc.returncode == 0, '`nnictl create` failed with code %d' % proc.returncode

In [14]:
t = time.time()
check_experiment_status()
time_tpe = time.time() - t

### 5. Show Results

The trial with the best metric and the corresponding metrics and hyperparameters can also be read from the Web UI

![](https://recodatasets.blob.core.windows.net/images/nni4.png)

or from the JSON file created by the training script. Below, we do this programmatically using [nni_utils.py](../../reco_utils/nni/nni_utils.py) 

In [15]:
check_metrics_written()
trials, best_metrics, best_params, best_trial_path = get_trials('maximize')

In [16]:
best_metrics

{'rmse': 0.9431428682250755,
 'ndcg_at_k': 0.06877421297298893,
 'precision_at_k': 0.06304579339723111}

In [17]:
best_params

{'parameter_id': 77,
 'parameter_source': 'algorithm',
 'parameters': {'n_factors': 10,
  'init_mean': 0.18324723602848197,
  'init_std_dev': 0.19895783184200538,
  'lr_bu': 0.0045632579026322175,
  'lr_bi': 0.0012889151584839513,
  'lr_pu': 0.024358724863615677,
  'lr_qi': 0.03678239608568307,
  'reg_bu': 0.35769103982372275,
  'reg_bi': 0.04206274699014475,
  'reg_pu': 0.303698430274383,
  'reg_qi': 0.13718916843606518}}

In [18]:
best_trial_path

'/home/anargyri/nni/experiments/vjYG4M0x/trials/UjNKs'

This directory path is where info about the trial can be found, including logs, parameters and the model that was learned. To evaluate the metrics on the test data, we get the SVD model that was saved as `model.dump` in the training script.

In [19]:
svd = surprise.dump.load(os.path.join(best_trial_path, "model.dump"))[1]

The following function computes all the metrics given an SVD model.

In [20]:
def compute_test_results(svd):
    test_results = {}
    predictions = compute_rating_predictions(svd, test, usercol="userID", itemcol="itemID")
    for metric in RATING_METRICS:
        test_results[metric] = eval(metric)(test, predictions)

    all_predictions = compute_ranking_predictions(svd, train, usercol="userID", itemcol="itemID", recommend_seen=RECOMMEND_SEEN)
    for metric in RANKING_METRICS:
        test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K)
    return test_results

In [21]:
test_results_tpe = compute_test_results(svd)
print(test_results_tpe)

{'rmse': 0.9365400832805513, 'precision_at_k': 0.06556503198294243, 'ndcg_at_k': 0.06982721829231966}


### 6. More Tuning Algorithms
We now apply other tuning algorithms supported by NNI to the same problem. For details about these tuners, see the [NNI docs.](https://nni.readthedocs.io/en/latest/tuners.html#)
The only change needed is in the relevant entry in the configuration file.

In [20]:
def stop_and_restart():
    proc = subprocess.run([sys.prefix + '/bin/nnictl', 'stop'])
    check_stopped()
    proc = subprocess.run([sys.prefix + '/bin/nnictl', 'create', '--config', config_path])
    assert proc.returncode == 0, '`nnictl create` failed with code %d' % proc.returncode
    check_experiment_status()

In [21]:
# Random search
config['tuner']['builtinTunerName'] = 'Random'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [22]:
t = time.time()
stop_and_restart()
time_random = time.time() - t

In [24]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_random = compute_test_results(svd)

In [25]:
# Annealing
config['tuner']['builtinTunerName'] = 'Anneal'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [26]:
t = time.time()
stop_and_restart()
time_anneal = time.time() - t

In [27]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_anneal = compute_test_results(svd)

In [28]:
# Naive evolutionary search
config['tuner']['builtinTunerName'] = 'Evolution'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [36]:
t = time.time()
stop_and_restart()
time_evolution = time.time() - t

In [37]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_evolution = compute_test_results(svd)

The SMAC tuner requires to have been installed with the following command <br>
`nnictl package install --name=SMAC`

In [38]:
# SMAC
config['tuner']['builtinTunerName'] = 'SMAC'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [39]:
t = time.time()
stop_and_restart()
time_smac = time.time() - t

In [40]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_smac = compute_test_results(svd)

In [41]:
# Metis
config['tuner']['builtinTunerName'] = 'MetisTuner'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [43]:
t = time.time()
stop_and_restart()
time_metis = time.time() - t

In [44]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_metis = compute_test_results(svd)

Hyperband follows a different style of configuration from other tuners. See [the NNI documentation](https://nni.readthedocs.io/en/latest/hyperbandAdvisor.html). Note that the [training script](../../reco_utils/nni/svd_training.py) needs to be adjusted as well, since each Hyperband trial receives an additional parameter `STEPS`, which corresponds to the resource allocation _r<sub>i</sub>_ in the [Hyperband algorithm](https://arxiv.org/pdf/1603.06560.pdf). In this example, we used `STEPS` in combination with `R` to determine the number of epochs that SVD will run for in every trial.

In [45]:
# Hyperband
config['advisor'] = {
  'builtinAdvisorName': 'Hyperband',
  'classArgs': {
    'R': NUM_EPOCHS,
    'eta': 3,
    'optimize_mode': 'maximize'
  }
}
config.pop('tuner')
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [54]:
t = time.time()
stop_and_restart()
time_hyperband = time.time() - t

In [55]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_hyperband = compute_test_results(svd)

In [75]:
test_results_tpe.update({'time': time_tpe})
test_results_random.update({'time': time_random})
test_results_anneal.update({'time': time_anneal})
test_results_evolution.update({'time': time_evolution})
test_results_smac.update({'time': time_smac})
test_results_metis.update({'time': time_metis})
test_results_hyperband.update({'time': time_hyperband})

In [76]:
res_df = pd.DataFrame(index=['TPE', 'Random Search', 'Annealing', 'Evolution', 'SMAC', 'Metis', 'Hyperband'],
                      data=[res for res in [test_results_tpe, test_results_random, test_results_anneal, test_results_evolution, 
                                            test_results_smac, test_results_metis, test_results_hyperband]] 
                     )

In [82]:
res_df.sort_values(by='precision_at_k', ascending=False).round(3)

,ndcg_at_k,precision_at_k,rmse,time
Annealing,0.127,0.107,1.050,526.314
SMAC,0.114,0.098,1.013,546.359
Random Search,0.063,0.059,1.091,526.244
TPE,0.056,0.052,0.936,520.578
Hyperband,0.055,0.051,0.993,325.190
Metis,0.047,0.042,0.984,886.680
Evolution,0.038,0.039,0.947,546.008


In [22]:
# Cleaning up resources
shutil.rmtree(TMP_DIR, ignore_errors=True)
shutil.rmtree(os.path.normpath(best_trial_path + '/../../..'), ignore_errors=True)

In [ ]:
# Record results with papermill for tests
pm.record("results", res_df.to_dict())

### 7. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the NNI toolkit. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas each of the above experiments took about 10. With NNI, one can take advantage of concurrency and multiple processors on a virtual machine and can use a variety of methods to navigate efficiently through a large space of hyperparameters.

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Neural Network Intelligence toolkit](https://github.com/Microsoft/nni)